# Sessie 05 - Opdracht Decision Trees 

In [13]:
%matplotlib inline
import datetime
import calendar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from scipy.stats import randint 
from scipy.stats import uniform
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier


pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

## Oefening 1. Energy consumption

In een goed geïsoleerd huis werden gedurende enkele maanden om de 10 minuten temperatuur- (in graden Celcius uitgedrukt) en vochtigheidsmetingen (in % uitgedrukt) gedaan. Er is ook weerdata beschikbaar van een nabij weerstation.
Tevens werd het elektrisch verbruik van de verlichting in het huis opgemeten samen met het elektrisch verbruik van andere toestellen.
De variabelen in de dataset hebben volgende betekenis:

- date time jaar-maand-dag uur:minuten:seconden
- Appliances: energieverbruik in Wh van elektrische toestellen in het huis
- lights: totaal energieverbruik van de verlichting in het huis in Wh
- T1: temperatuur in de keuken
- RH_1: vochtigheidsgraad in de keuken
- T2:  temperatuur in de living
- RH_2: vochtigheidsgraad in de living
- T3: temperatuur in de wasruimte
- RH_3: vochtigheidsgraad in de wasruimte 
- T4 temperatuur in het bureau
- RH_4: vochtigheidsgraad in het bureau 
- T5: temperatuur in de badkamer
- RH_5: vochtigheidsgraad in de badkamer
- T6: gemeten buitentemperatuur aan de noordkant van het huis
- RH_6: gemeten vochtigheidsgraad buiten aan de noordkant van het huis
- T7: temperatuur in de strijkruimte
- RH_7: vochtigheidsgraad in de strijkruimte
- T8: Temperatuur in kinderkamer 2
- RH_8: Vochtigheidsgraad in kindkamer 2
- T9: Temperatuur in de slaapkamer van de ouders
- RH_9: Vochtigheidsgraad in de slaapkamer van de ouders
- To: buitentemperatuur gemeten door het weerstation in de buurt
- Pressure: luchtdruk in mm Hg gemeten door het weerstation in de buurt
- RH_out: Vochtigheid buiten, gemeten door het weerstation in de buurt
- Windspeed: windsnelheid in m/s, gemeten door het weerstation in de buurt.
- Visibility: de zichtbaarheid in km, opgetekend door het weestation in de buurt.
- Tdewpoint: het dauwpunt gemeten door het weerstation in de buurt
- rv1: random variabele 1
- rv2, Random variable 2

De twee random variabelen rv1 en rv2 mogen uit de dataset worden verwijderd.

De bedoeling van deze opdracht is om een regressiemodel te trainen dat zo precies mogelijk het elektriciteitsverbruik van de toestellen (Appliances) kan voorspellen uit de andere variabelen.

In [3]:
dataset = pd.read_csv('Energy_consumption.csv')
dataset.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


### Uitwerking

- Probeer eerst het elektriciteitsverbruik zo nauwkeurig mogelijk te voorspellen uit de andere variabelen gebruik makende van lineaire regressie, zoals je hebt gedaan in de sessie rond lineaire regressie.

- Train nu ook een Random Forest Regressor en optimaliseer deze via hyperparameter tuning: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html


Je krijgt alle vrijheid in welke technieken je toepast. Jouw enige doel is een zo hoog mogelijke R²-score te behalen op een test set bestaande uit 5000 samples. Probeer ook zo goed mogelijk de datum en tijdstippen te gebruiken.

Bepaal uit de gertrainde Random Forest Classifier de belangrijkheid van de features (model.feature_importances_).
Welke zijn de 5 belangrijkste features?

#### Uitbreiding

Aangezien de data in de tijd evolueert kunnen we die beschouwen als een tijdreeks. Daarom kan het interessant zijn om te onderzoeken of er informatie uit de opeenvolging van de metingen te halen valt.

Bekijk daarom of je betere predicties kunt maken wanneer je meerdere opeenvolgende metingen gebruikt om het elektriciteitsverbruik te voorspellen.
Dit betekent dat je een nieuwe training set en test set dient aan te maken waarbij elke rij de oorspronkelijk features bevat aangevuld met de features van aantal voorgaande rijen.
Bijvoorbeeld wanneer je het elektriciteitsverbruik wilt voorspellen op basis van 3 opeenvolgende metingen dan bevat elke rij van X (later op te splitsen in X_train en X_test) de features van het huidige tijdstip, maar ook de features van het voorgaande tijdstip en het tijdstip daarvoor. Dit betekent dat in dit voorbeeld het aantal kolommen van X drie keer groter is geworden.

Probeer dit en bespreek of dit betere predicties oplevert.


In [ ]:
# Uitwerking


## Oefening 2 - NASA
Gegeven is de dataset **nasa.csv**. Deze dataset bevat gegevens en metingen van waargenomen meteorieten. De bedoeling is om op basis van deze metingen (features) een betrouwbare voorspelling te kunnen doen of een bepaalde meteoriet al dan niet gevaarlijk is. Of ze met andere woorden kan inslaan op de aarde.
Er wordt een accuracy van minstens 95% geëist.

### Inlezen en preprocessing van de dataset

1.  Verwijder de kolommen 'Neo Reference ID', 'Name', 'Close Approach Date','Epoch Date Close Approach', 'Orbiting Body','Orbit Determination Date' en 'Equinox'
2. Verwijder de samples met ontbrekende waarden volledig(listwise deletion)
3. Kuis de data op waar nodig
4. Vervang de waarden uit de output kolom Hazardous: False / True -> 0 / 1
5. Voer one-hot encoding uit op de categorische variabelen

In [5]:
# Inladen van de dataset
dataset = pd.read_csv('nasa.csv')
dataset.head()

,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Close Approach Date,Epoch Date Close Approach,Relative Velocity km per sec,Relative Velocity km per hr,Miss Dist.(Astronomical),Miss Dist.(lunar),Miss Dist.(kilometers),Orbiting Body,Orbit ID,Orbit Determination Date,Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Epoch Osculation,Eccentricity,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,3703080,3703080,21.6,0.127220,0.284472,1995-01-01,788947200000,6.115834,22017.003799,0.419483,163.178711,62753692.0,Earth,17,2017-04-06 08:36:37,5,0.025282,4634,2458000.5,0.425549,1.407011,6.025981,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,J2000,yes
1,3723955,3723955,21.3,0.146068,0.326618,1995-01-01,788947200000,18.113985,65210.346095,0.383014,148.992630,57298148.0,Earth,21,2017-04-06 08:32:49,3,0.186935,5457,2458000.5,0.351674,1.107776,28.412996,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,J2000,no
2,2446862,2446862,20.3,0.231502,0.517654,1995-01-08,789552000000,7.590711,27326.560182,0.050956,19.821890,7622911.5,Earth,22,2017-04-06 09:20:19,0,0.043058,4557,2458000.5,0.348248,1.458824,4.237961,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,J2000,yes
3,3092506,3092506,27.4,0.008801,0.019681,1995-01-15,790156800000,11.173874,40225.948191,0.285322,110.990387,42683616.0,Earth,7,2017-04-06 09:15:49,6,0.005512,5093,2458000.5,0.216578,1.255903,7.905894,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,J2000,no
4,3514799,3514799,21.6,0.127220,0.284472,1995-01-15,790156800000,9.840831,35426.991794,0.407832,158.646713,61010824.0,Earth,25,2017-04-06 08:57:58,1,0.034798,5154,2458000.5,0.210448,1.225615,16.793382,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,J2000,yes


In [6]:
# Verwijderen van de gevraagde kolommen


# Vervang de waarden uit de output kolom Hazardous: False / True -> 0 / 1


# Verwijderen van samples met ontbrekende waarden


# Verwijderen van samples met uitschieters en onrealistische waarden

# One-hot encoding


Controleer of de dataset al dan niet **gebalanceerd** is. **Antwoord** duidelijk of dit al dan niet het geval is en wat de **mogelijke gevolgen** kunnen zijn mocht deze niet gebalanceerd zijn.

In [7]:
# Controleer of de dataset gebalanceerd is. 


# Antwoord:

### Trainen van de classifier

Splits eerst de features op in een training- en test set. 

Zorg ervoor dat er exact **1000 samples** in de test set steken en gebruik een **random_state = 0**.

In [8]:
# Opsplitsen in features en targets. 


# Opsplitsen in test en training_set met 1000 waarden in test set en random_state = 0. Normaliseer de features.


Train zowel een logistic regression en SVM classifier. Je mag onmiddellijk het optimale model zoeken via cross-validation. Test deze classifiers: bepaal de accuracy, precision, recall, f1-score en ROC (waar mogelijk)

In [10]:
# Trainen en testen van een logistic regression en SVM classifier


Train een **Random Forest Classifier**. 
- Test deze classifier op de trainingset. Wat zijn jouw bevindingen en conclusies? Vergelijk de resultaten met deze van logistic regression en SVM.

Een lijst van hyperparameters zijn te vinden via https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

- Bepaal via deze random forest tree classifier de belangrijkste features. 

In [11]:
# Trainen en testen van een random forest tree classifier


Bespreek in welke mate de verschillende hyperparameters kunnen leiden tot overfitting/underfitting.

ANTWOORD:

Bepaal uit de gertrainde Random Forest Classifier de belangrijkheid van de features (model.feature_importances_). 

Welke zijn de 10 belangrijkste features?

Train het model opnieuw met deze 10 belangrijkste features. Test het model en verklaar de resulaten.

Reduceer het aantal features nog verder en bekijk telkens de accuracy. Verklaar de resultaten.

In [12]:
# Reduceren van het aantal features en hertrainen via een random forest classifier.


Kijk of je via boosting (Adaboost) de accuracy op de test set nog kunt verhogen. Het type classifier (base estimator) mag je zelf kiezen.